# obtain and curate DR3 binary solutions 

### 2023-10-10 - 2024-04-17 Johannes Sahlmann

- download data from Gaia archive

# Environment creation

```
conda config --append channels conda-forge
conda create --name gorbit --yes python=3.9 pip ipykernel jupyterlab git git-lfs pandas astropy astroquery numpy matplotlib pyarrow uncertainties  
conda activate gorbit
ipython kernel install --user --name=gorbit
pip install pystrometry   
```

## Install gaia-ad
```
git clone https://gitlab.esa.int/gaia-agis/machine-learning/gaia-anomaly-detection
cd gaia-anomaly-detection/
pip install -e .
```


In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import copy
from collections import OrderedDict
import glob
import logging
import os
import numpy as np
import pandas as pd

# import astropy.units as u
# from astropy.table import Table
# from astropy.time import Time
# from astroquery.gaia import Gaia, TapPlus
# import matplotlib as mp
import matplotlib.pyplot as pl

### Import pystrometry (make sure its version is >=0.3.0)

In [2]:
import pystrometry
print(pystrometry.__version__)

0.4.3


In [3]:
import pyarrow
print(pyarrow.__version__)

14.0.2


In [4]:
from pystrometry.utils import du437_tools
from pystrometry import gaia_astrometry
from pystrometry.utils.archives import get_gaiadr_data


universal_helpers not available


In [5]:
# dataset_tag = 'v0.0'
# dataset_tag = 'v0.1'
dataset_tag = 'v0.6'

out_path = f'/media/team_workspaces/Gaia-Orbit-Classification/sandbox/dataset_{dataset_tag}'
os.makedirs(out_path, exist_ok=True)

In [6]:
# overwrite = True
overwrite = False
overwrite_archive_query = False

In [7]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [8]:
os.getcwd()

'/media/home/gaia-anomaly-detection/notebooks/jsahlmann/ml_gaia_paper_mnras'

## Set the output path for data and figures. Customise this cell if desired. 

In [9]:
root_data_dir = '/media/team_workspaces/Gaia-Orbit-Classification/jsahlmann/data'

In [10]:
input_list_name = 'nss'
gaia_data_release = 'gaiadr3'

data_dir = os.path.join(root_data_dir, gaia_data_release)

# set folder to save figures
plot_dir = os.path.join(data_dir, 'figures', input_list_name)

for dir in [data_dir, plot_dir]:
    os.makedirs(dir, exist_ok=True)
    


## retrieve all 'nss_two_body_orbit' solutions 

If one needs access to the `corr_vec` field, e.g. to compute uncertainties in the geometric parameters, one has to query the full table from the archive. Using the `source_id_array` argument of the `get_gaiadr_data` function invokes an inner join operation in GACS and the returned table does not have the `corr_vec` field. The reason for this behaviour is unknown, to be investigated and reported to DPAC if applicable.

In [11]:
gaia_table_name = 'nss_two_body_orbit'
analysis_dataset_name = '{}_all'.format(gaia_table_name)
nss_all = get_gaiadr_data(analysis_dataset_name, data_dir, gaia_data_release=gaia_data_release, gaia_table_name=gaia_table_name, overwrite_query=overwrite_archive_query, source_id_array=None)
logging.info('Dataset has {} unique source_ids, i.e. {} duplicate source_ids'.format(len(nss_all['source_id'].unique()), len(nss_all)-len(nss_all['source_id'].unique())))

INFO:root:Dataset has 437275 unique source_ids, i.e. 5930 duplicate source_ids


Retrieved 443205 rows from gaiadr3.nss_two_body_orbit


In [12]:
nss_all['nss_solution_type'].unique()

array(['Orbital', 'SB1', 'EclipsingBinary', 'SB2C', 'AstroSpectroSB1',
       'OrbitalAlternative', 'SB2', 'OrbitalTargetedSearchValidated',
       'EclipsingSpectro', 'OrbitalTargetedSearch', 'SB1C',
       'OrbitalAlternativeValidated'], dtype=object)

# Select only astrometric orbits

In [13]:
selected = nss_all[nss_all['nss_solution_type'].isin(['Orbital', 'AstroSpectroSB1','OrbitalAlternative', 'OrbitalTargetedSearchValidated', 'OrbitalTargetedSearch', 'OrbitalAlternativeValidated'])].copy() #, 'AstroSpectroSB1'

print(len(selected))

169227


In [14]:
selected['parallax'].min()

0.10746699584676161

In [15]:
# selected['parallax'].hist(bins=100, log=True, range=(0.1, 1))

In [16]:
logging.info('Dataset has {} unique source_ids, i.e. {} duplicate source_ids'.format(len(selected['source_id'].unique()), len(selected)-len(selected['source_id'].unique())))

INFO:root:Dataset has 169129 unique source_ids, i.e. 98 duplicate source_ids


# remove sources with both AstroSpectroSB1 and OrbitalTargetedSearch solution (oversight of DR3), keep AstroSpectroSB1

In [17]:
n_occurrence = selected['source_id'].value_counts()==2
duplicated_source_ids = n_occurrence[n_occurrence].index

selected[selected['source_id'].isin(duplicated_source_ids)]['nss_solution_type'].value_counts()

nss_solution_type
AstroSpectroSB1                   98
OrbitalTargetedSearchValidated    92
OrbitalTargetedSearch              6
Name: count, dtype: int64

In [18]:
logging.info('Dataset has {} unique source_ids, i.e. {} duplicate source_ids'.format(len(selected['source_id'].unique()), len(selected)-len(selected['source_id'].unique())))

solutions_to_drop = (selected['source_id'].isin(duplicated_source_ids)) & (selected['nss_solution_type']!='AstroSpectroSB1')
selected = selected[~solutions_to_drop]
logging.info('Dataset has {} unique source_ids, i.e. {} duplicate source_ids'.format(len(selected['source_id'].unique()), len(selected)-len(selected['source_id'].unique())))
assert len(selected) == selected['source_id'].nunique()


INFO:root:Dataset has 169129 unique source_ids, i.e. 98 duplicate source_ids
INFO:root:Dataset has 169129 unique source_ids, i.e. 0 duplicate source_ids


# Add geometric orbital elements to dataframe

In [19]:
assert 'corr_vec' in selected.columns
selected = selected.nss.add_geometric_elements()
selected = selected.nss.add_geometric_elements_unc()

Columns already exist!


# add mass function

In [20]:
import astropy.units as u
def mass_function_msun(semimajor_axis_mas, period_day, parallax_mas):
    """Return the value of the mass function in M_sun, see Eq. 13 in Halbwachs+23."""
    return semimajor_axis_mas**3 * (u.year.to(u.day))**2 / (period_day**2 * parallax_mas**3)


selected['mass_function_msun'] = mass_function_msun(selected['p1_a1_mas'].values, selected['period'].values, selected['parallax'].values)

# selected['mass_function_msun'].plot(kind='hist', histtype='step', logx=True, bins=np.logspace(-4,1,100), lw=2)

In [21]:
len(selected)

169129

In [22]:
selected.columns

Index(['solution_id', 'source_id', 'nss_solution_type', 'ra', 'ra_error',
       'dec', 'dec_error', 'parallax', 'parallax_error', 'pmra', 'pmra_error',
       'pmdec', 'pmdec_error', 'a_thiele_innes', 'a_thiele_innes_error',
       'b_thiele_innes', 'b_thiele_innes_error', 'f_thiele_innes',
       'f_thiele_innes_error', 'g_thiele_innes', 'g_thiele_innes_error',
       'c_thiele_innes', 'c_thiele_innes_error', 'h_thiele_innes',
       'h_thiele_innes_error', 'period', 'period_error', 't_periastron',
       't_periastron_error', 'eccentricity', 'eccentricity_error',
       'center_of_mass_velocity', 'center_of_mass_velocity_error',
       'semi_amplitude_primary', 'semi_amplitude_primary_error',
       'semi_amplitude_secondary', 'semi_amplitude_secondary_error',
       'mass_ratio', 'mass_ratio_error', 'fill_factor_primary',
       'fill_factor_primary_error', 'fill_factor_secondary',
       'fill_factor_secondary_error', 'inclination', 'inclination_error',
       'arg_periastron', 'a

In [23]:
overwrite

False

In [24]:
nns_table_columns_to_export = ['source_id', 'nss_solution_type', 'ra', 'dec', 'parallax', 'parallax_error', 'pmra', 'pmdec']
nns_table_columns_to_export += ['period', 'period_error', 't_periastron', 't_periastron_error', 'eccentricity', 'eccentricity_error']
nns_table_columns_to_export += ['astrometric_n_good_obs_al', 'obj_func', 'goodness_of_fit', 'efficiency', 'significance']

nns_table_columns_to_export_original = copy.deepcopy(nns_table_columns_to_export)
nns_table_columns_to_export_computed = ['p1_a1_mas', 'p1_omega_deg', 'p1_OMEGA_deg', 'p1_incl_deg', 'p1_period_day']
nns_table_columns_to_export_computed += ['mass_function_msun']
nns_table_columns_to_export += nns_table_columns_to_export_computed


# fill NaNs in some columns
selected['has_filled_nans'] = 0
nns_table_columns_to_export += ['has_filled_nans']

sel_index = selected['eccentricity_error'].isna()
print(f"'eccentricity_error' {sum(sel_index)}")
selected.loc[sel_index, 'eccentricity_error'] = 0.
selected.loc[sel_index, 'has_filled_nans'] = 1


# display(selected[nns_table_columns_to_export].isna().any())
# display(selected[nns_table_columns_to_export].info())

logging.info('Dataset has {} unique source_ids, i.e. {} duplicate source_ids'.format(len(selected['source_id'].unique()), len(selected)-len(selected['source_id'].unique())))
assert len(selected) == selected['source_id'].nunique()

outfile = os.path.join(out_path, 'nss_two_body_orbit_astrometric_orbits.parquet')
if (overwrite) or (~os.path.isfile(outfile)):
    selected[nns_table_columns_to_export].to_parquet(outfile)
    logging.info(f"Wrote {outfile}")

    
output='latex'    
    
doc = pd.DataFrame()
doc['column_name']=nns_table_columns_to_export
nns_table_columns_to_export_latex = [s.replace('_', '\_') for s in nns_table_columns_to_export]
doc['column_name_latex'] = nns_table_columns_to_export_latex
# doc['comment'] = ''

def nss_doc_link(col):
    if col in nns_table_columns_to_export_original:
        if output=='latex':
            return f"\\href{{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_non--single_stars_tables/ssec_dm_nss_two_body_orbit.html#nss_two_body_orbit-{col}}}{{online documentation}}"
        else:
            return f"[online documentation](https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_non--single_stars_tables/ssec_dm_nss_two_body_orbit.html#nss_two_body_orbit-{col})"
    else:
        return ""

doc['description'] = doc['column_name'].apply(nss_doc_link) 

doc.loc[doc['column_name'] == 'p1_a1_mas', 'description'] = '$a_0$ (mas)'
doc.loc[doc['column_name'] == 'p1_omega_deg', 'description'] = '$\\omega$ (deg)'
doc.loc[doc['column_name'] == 'p1_OMEGA_deg', 'description'] = '$\\Omega$ (deg)'
doc.loc[doc['column_name'] == 'p1_incl_deg', 'description'] = '$i$ (deg)'
doc.loc[doc['column_name'] == 'mass_function_msun', 'description'] = '$f_M$ ($M_\\sun$)'
# doc.loc[doc['column_name'] == 'p1_a1_mas', 'description'] = 'semimajor axis of the astrometric orbit (mas)'
# doc.loc[doc['column_name'] == 'p1_omega_deg', 'description'] = 'Argument of periastron (deg)'
# doc.loc[doc['column_name'] == 'p1_OMEGA_deg', 'description'] = 'Ascending node (deg)'
# doc.loc[doc['column_name'] == 'p1_incl_deg', 'description'] = 'Inclination (deg), 90deg is edge-on'
# doc.loc[doc['column_name'] == 'p1_period_day', 'description'] = 'copy of `period`'

doc['comment'] = ''
for c in ['parallax', 'pmra', 'pmdec']:
    doc.loc[doc['column_name'] == c, 'comment'] = 'confounder'
for c in ['nss_solution_type', 'source_id', "astrometric_primary_flag"]:
    doc.loc[doc['column_name'] == c, 'comment'] = 'not used'
for c in ['p1_a1_mas', 'p1_omega_deg', 'p1_OMEGA_deg', 'p1_incl_deg', 'mass_function_msun']:
    doc.loc[doc['column_name'] == c, 'comment'] = 'computed'


doc.loc[doc['column_name'] == 'p1_a1_mas', 'column_name_latex'] = 'p1\_a0\_mas' 
# doc.rename(columns={'p1_a1_mas': 'p1_a0_mas'}, inplace=True)    
doc = doc[~doc['column_name'].isin(['has_filled_nans', 'p1_period_day'])]    
    
print(doc[['column_name_latex', 'description', 'comment']].to_latex(index=False))
# print(doc.to_markdown())

INFO:root:Dataset has 169129 unique source_ids, i.e. 0 duplicate source_ids


'eccentricity_error' 1979


INFO:root:Wrote /media/team_workspaces/Gaia-Orbit-Classification/sandbox/dataset_v0.6/nss_two_body_orbit_astrometric_orbits.parquet


\begin{tabular}{lll}
\toprule
column_name_latex & description & comment \\
\midrule
source\_id & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_non--single_stars_tables/ssec_dm_nss_two_body_orbit.html#nss_two_body_orbit-source_id}{online documentation} & not used \\
nss\_solution\_type & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_non--single_stars_tables/ssec_dm_nss_two_body_orbit.html#nss_two_body_orbit-nss_solution_type}{online documentation} & not used \\
ra & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_non--single_stars_tables/ssec_dm_nss_two_body_orbit.html#nss_two_body_orbit-ra}{online documentation} &  \\
dec & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_non--single_stars_tables/ssec_dm_nss_two_body_orbit.html#nss_two_body_orbit-dec}{online documentation} &  \\
parallax & \href{https://ge

In [25]:
extra_columns = ['a_thiele_innes', 'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes']

# nns_table_columns_to_export_original


In [26]:
# selected[selected['source_id'] == 1215279005201870336]

In [27]:
# display(selected[selected['source_id'].isin([3230836376154368384, 3573773434382465280, 1055017523232004352, 6795834500861297408, 5557776367807067648, 4361277334741236480, 4869246209213679872])][nns_table_columns_to_export])

In [28]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', 100)
# selected[selected['source_id']==4364702279101280256].iloc[0]to_markdown

# Plot the cos(inclination) distribution 

In [29]:
# col = 'p1_incl_deg'
# np.cos(np.deg2rad(selected[col])).plot(kind='hist', log=True, bins=100)

# Query gaia_source (single star model results)

In [30]:
assert len(selected) == selected['source_id'].nunique()

analysis_dataset_name = 'astrometric_binaries'
source_id_array = selected['source_id'].values
# source_id_array = selected_source_id_array

gaia_table_name = 'gaia_source'
# overwrite_archive_query = True
gaia_source_selected = get_gaiadr_data(analysis_dataset_name, plot_dir, source_id_array, gaia_data_release, gaia_table_name=gaia_table_name, overwrite_query=overwrite_archive_query)
plot_dir

Retrieved 169129 rows from gaiadr3.gaia_source


'/media/team_workspaces/Gaia-Orbit-Classification/jsahlmann/data/gaiadr3/figures/nss'

In [31]:
gaia_source_selected.iloc[0]['ref_epoch']

2016.0

In [32]:
# gaia_source_selected.plot('phot_g_mean_mag', 'rv_method_used', kind='scatter')
# gaia_source_selected.plot('radial_velocity_error', 'rv_method_used', kind='scatter')
# gaia_source_selected.plot('phot_g_mean_mag', 'radial_velocity_error', c='rv_method_used', kind='scattera
# gaia_source_selected.plot('phot_g_mean_mag', 'radial_velocity_error', kind='hexbin', C='rv_method_used', cmap="Spectral", reduce_C_function=np.mean) # , color_by='rv_method_used'

In [33]:
gaia_source_selected['rv_method_used'].value_counts()/gaia_source_selected['rv_method_used'].count()

rv_method_used
2    0.56626
1    0.43374
Name: count, dtype: Float64

In [34]:
(selected['nss_solution_type'].isin(['AstroSpectroSB1'])).value_counts()/len(selected)

nss_solution_type
False    0.802121
True     0.197879
Name: count, dtype: float64

In [35]:
data_path = f'/media/team_workspaces/Gaia-Orbit-Classification/sandbox/dataset_{dataset_tag}'
outfile = os.path.join(data_path, 'labelled_sources.parquet')
labels = pd.read_parquet(outfile)
gaia_source_selected[gaia_source_selected['source_id'].isin(labels['source_id'])]['rv_method_used'].value_counts()
gaia_source_selected[gaia_source_selected['source_id'].isin(labels['source_id'])]['rv_method_used'].value_counts()/gaia_source_selected[gaia_source_selected['source_id'].isin(labels['source_id'])]['rv_method_used'].count()

rv_method_used
2    0.641723
1    0.358277
Name: count, dtype: Float64

In [36]:
gaia_source_columns_to_export = ['source_id', 'ra', 'dec', 'parallax', 'parallax_error', 'pmra', 'pmdec']
gaia_source_columns_to_export += ['visibility_periods_used', 'radial_velocity', 'radial_velocity_error', 'astrometric_chi2_al', 'ipd_gof_harmonic_amplitude', 'bp_rp', 'phot_bp_rp_excess_factor']
gaia_source_columns_to_export += ['ruwe', 'phot_g_mean_mag', 'phot_rp_mean_mag', 'phot_bp_mean_mag', 'astrometric_excess_noise_sig', 'astrometric_primary_flag', 'astrometric_excess_noise', 'astrometric_n_good_obs_al']


# compute absolute magnitude

In [37]:
gaia_source_selected = gaia_source_selected.nss.add_absolute_magnitude()
gaia_source_selected['has_filled_nans'] = 0
gaia_source_columns_to_export += ['absolute_phot_g_mean_mag']
gaia_source_columns_to_export += ['has_filled_nans']
gaia_source_columns_to_export

['source_id',
 'ra',
 'dec',
 'parallax',
 'parallax_error',
 'pmra',
 'pmdec',
 'visibility_periods_used',
 'radial_velocity',
 'radial_velocity_error',
 'astrometric_chi2_al',
 'ipd_gof_harmonic_amplitude',
 'bp_rp',
 'phot_bp_rp_excess_factor',
 'ruwe',
 'phot_g_mean_mag',
 'phot_rp_mean_mag',
 'phot_bp_mean_mag',
 'astrometric_excess_noise_sig',
 'astrometric_primary_flag',
 'astrometric_excess_noise',
 'astrometric_n_good_obs_al',
 'absolute_phot_g_mean_mag',
 'has_filled_nans']

In [38]:
# doc = pd.DataFrame()
# doc['column_name']=gaia_source_columns_to_export

# def nss_doc_link(col):
#     if col in gaia_source_columns_to_export:
#         return f"[online documentation](https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-{col})"
#     else:
#         return ""

# doc['description'] = doc['column_name'].apply(nss_doc_link) 

# doc.loc[doc['column_name'] == 'absolute_phot_g_mean_mag', 'description'] = 'Estimated absolute G magnitude (mag)'
# doc.loc[doc['column_name'] == 'has_filled_nans', 'description'] = "For this source a NaN values had beed filled with a 'neutral' value."

# # doc.loc[doc['column_name'] == 'p1_omega_deg', 'description'] = 'Argument of periastron (deg)'
# # doc.loc[doc['column_name'] == 'p1_OMEGA_deg', 'description'] = 'Ascending node (deg)'
# # doc.loc[doc['column_name'] == 'p1_incl_deg', 'description'] = 'Inclination (deg), 90deg is edge-on'
# # doc.loc[doc['column_name'] == 'p1_period_day', 'description'] = 'copy of `period`'
# print(doc.to_markdown())



output='latex'    
    
doc = pd.DataFrame()
doc['column_name']=gaia_source_columns_to_export
gaia_source_columns_to_export_latex = [s.replace('_', '\_') for s in gaia_source_columns_to_export]
doc['column_name_latex'] = gaia_source_columns_to_export_latex


def nss_doc_link(col):
    if col in gaia_source_columns_to_export:
        if output=='latex':
            return f"\\href{{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-{col}}}{{online documentation}}"
        else:
            return f"[online documentation](https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-{col})"
    else:
        return ""

doc['description'] = doc['column_name'].apply(nss_doc_link) 

doc.loc[doc['column_name'] == 'absolute_phot_g_mean_mag', 'description'] = 'absolute $G$ mag'

doc['comment'] = ''
for c in ['parallax', 'pmra', 'pmdec', "phot_g_mean_mag"]:
    doc.loc[doc['column_name'] == c, 'comment'] = 'confounder'
for c in ['source_id', "astrometric_primary_flag"]:
    doc.loc[doc['column_name'] == c, 'comment'] = 'not used'
for c in ['absolute_phot_g_mean_mag']:
    doc.loc[doc['column_name'] == c, 'comment'] = 'computed'

doc = doc[~doc['column_name'].isin(['has_filled_nans', 'p1_period_day'])]    
    
print(doc[['column_name_latex', 'description', 'comment']].to_latex(index=False))




\begin{tabular}{lll}
\toprule
column_name_latex & description & comment \\
\midrule
source\_id & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-source_id}{online documentation} & not used \\
ra & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-ra}{online documentation} &  \\
dec & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-dec}{online documentation} &  \\
parallax & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html#gaia_source-parallax}{online documentation} & confounder \\
parallax\_error & \href{https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_ma

# fill NaNs with neutral values

In [39]:
sel_index = gaia_source_selected['radial_velocity_error'].isna()
print(f"'radial_velocity_error' {sum(sel_index)}")
gaia_source_selected.loc[sel_index, 'radial_velocity_error'] = gaia_source_selected['radial_velocity_error'].median()
print(f"'radial_velocity_error' value {gaia_source_selected['radial_velocity_error'].median()}")

gaia_source_selected.loc[sel_index, 'has_filled_nans'] = 1

sel_index = gaia_source_selected['radial_velocity'].isna()
print(f"'radial_velocity' {sum(sel_index)}")
gaia_source_selected.loc[sel_index, 'radial_velocity'] = 0.
gaia_source_selected.loc[sel_index, 'has_filled_nans'] = 1

for col in ['phot_rp_mean_mag', 'phot_bp_mean_mag', 'bp_rp', 'phot_bp_rp_excess_factor']:
    sel_index = gaia_source_selected[col].isna()
    print(f"{col} {sum(sel_index)}")
    gaia_source_selected.loc[sel_index, col] = gaia_source_selected[col].median()
    print(f"{col} value {gaia_source_selected[col].median()}")
    gaia_source_selected.loc[sel_index, 'has_filled_nans'] = 1

# sel_index = gaia_source_selected[].isna()
# gaia_source_selected.loc[sel_index, 'phot_bp_mean_mag'] = gaia_source_selected['phot_bp_mean_mag'].median()
# gaia_source_selected.loc[sel_index, 'has_filled_nans'] = 1



'radial_velocity_error' 21460
'radial_velocity_error' value 2.4064080715179443
'radial_velocity' 21460
phot_rp_mean_mag 2
phot_rp_mean_mag value 12.675403594970703
phot_bp_mean_mag 2
phot_bp_mean_mag value 13.704318046569824
bp_rp 2
bp_rp value 0.9935998916625977
phot_bp_rp_excess_factor 2
phot_bp_rp_excess_factor value 1.2160660028457642


In [40]:
gaia_source_selected[gaia_source_columns_to_export].isna().any()

source_id                       False
ra                              False
dec                             False
parallax                        False
parallax_error                  False
pmra                            False
pmdec                           False
visibility_periods_used         False
radial_velocity                 False
radial_velocity_error           False
astrometric_chi2_al             False
ipd_gof_harmonic_amplitude      False
bp_rp                           False
phot_bp_rp_excess_factor        False
ruwe                            False
phot_g_mean_mag                 False
phot_rp_mean_mag                False
phot_bp_mean_mag                False
astrometric_excess_noise_sig    False
astrometric_primary_flag        False
astrometric_excess_noise        False
astrometric_n_good_obs_al       False
absolute_phot_g_mean_mag        False
has_filled_nans                 False
dtype: bool

In [41]:
print(len(gaia_source_selected)-gaia_source_selected['source_id'].nunique())

0


In [42]:
assert len(gaia_source_selected) == gaia_source_selected['source_id'].nunique()

outfile = os.path.join(out_path, 'gaia_source_astrometric_orbits.parquet')
if (overwrite) or (~os.path.isfile(outfile)):    
    gaia_source_selected[gaia_source_columns_to_export].to_parquet(outfile)
    logging.info(f"Wrote {outfile}")


INFO:root:Wrote /media/team_workspaces/Gaia-Orbit-Classification/sandbox/dataset_v0.6/gaia_source_astrometric_orbits.parquet


In [43]:
gaia_source_selected['has_filled_nans'].value_counts()

has_filled_nans
0    147667
1     21462
Name: count, dtype: int64

In [44]:
gaia_source_selected[gaia_source_columns_to_export].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169129 entries, 0 to 169128
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   source_id                     169129 non-null  int64  
 1   ra                            169129 non-null  float64
 2   dec                           169129 non-null  float64
 3   parallax                      169129 non-null  float64
 4   parallax_error                169129 non-null  float32
 5   pmra                          169129 non-null  float64
 6   pmdec                         169129 non-null  float64
 7   visibility_periods_used       169129 non-null  int16  
 8   radial_velocity               169129 non-null  float32
 9   radial_velocity_error         169129 non-null  float32
 10  astrometric_chi2_al           169129 non-null  float32
 11  ipd_gof_harmonic_amplitude    169129 non-null  float32
 12  bp_rp                         169129 non-nul

In [45]:

# gaia_source_selected.hist('phot_rp_mean_mag', bins=100)